In [1]:
import datetime as dt
from utility import (
    read_sql,
    fd_alive_funds,
    fd_basicinfo,
    fd_typeclass,
    fd_hshkiport,
    fd_assetportfolio,
    fd_derieden,
    nearest_report_date
)
import pandas as pd
from PyFin.api import makeSchedule
from PyFin.api import BizDayConventions

In [2]:
# 获取回溯的报告日

today = dt.datetime.today()
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [3]:
# 获取所有目标债券型基金（清算结束日期晚于当前日）

security_ids = fd_alive_funds(report_dates_begin, 3)

In [4]:
# 获取相关组合情况

asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).fillna(0)
asset_port["CONVBDRTO_TO_BDRTO"] = asset_port["CONVBDRTO"] / asset_port["BDRTO"] * 100

In [5]:
last_portfolio = asset_port.groupby("SECURITYID").last()[["BDRTO"]]
last_4_cov_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["CONVBDRTO_TO_BDRTO"]]
last_4_stk_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["EQUITYINVERTO"]]

In [6]:
agg_port = pd.concat([last_portfolio, last_4_cov_portfolio, last_4_stk_portfolio], axis=1).reset_index()
agg_port = pd.merge(agg_port, fd_typeclass(agg_port.SECURITYID.tolist(), current_date))
basic_info = fd_basicinfo(agg_port.SECURITYID.tolist())
agg_port = pd.merge(agg_port, basic_info)

In [7]:
agg_port.L3NAME.unique()

array(['普通债券型基金(一级A类)', '普通债券型基金(一级B/C类)', '普通债券型基金(二级A类)',
       '普通债券型基金(二级B/C类)', '中短期标准债券型基金', '普通债券型基金(可投转债A类)',
       '普通债券型基金(可投转债B类)', '指数债券型基金(A类)', '长期标准债券型基金(A类)', '可转换债券型基金(A类)',
       '可转换债券型基金(B/C类)', '债券型分级子基金(优先份额)', '债券型分级子基金(进取份额)',
       '指数债券型基金(B/C类)', '长期标准债券型基金(B/C类)', '短期理财债券型基金(A类)',
       '短期理财债券型基金(B/C类)'], dtype=object)

In [8]:
# 分类
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port.L2NAME == "短期理财债券型基金"), "债券型（子类）"] = "短期纯债型基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 80), "债券型（子类）"] = "可转债基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 5) & (agg_port["CONVBDRTO_TO_BDRTO"] < 80) & (agg_port["EQUITYINVERTO"] <= 0.1), "债券型（子类）"] = "可投转债型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["EQUITYINVERTO"] >= 0.1) & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "可投股票型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["L2NAME"] == "指数债券型基金") & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "被动指数型债券基金"
agg_port.loc[pd.isnull(agg_port["债券型（子类）"]), "债券型（子类）"] = "中长期纯债型基金"

In [9]:
agg_port.groupby("债券型（子类）").count()

,SECURITYID,BDRTO,CONVBDRTO_TO_BDRTO,EQUITYINVERTO,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE
债券型（子类）,,,,,,,,,,,,,,,
中长期纯债型基金,638,638,625,632,638,638,638,638,638,638,638,450,638,638,638
可投股票型债券基金,370,370,370,370,370,370,370,370,370,370,370,321,370,370,370
可投转债型债券基金,87,87,87,87,87,87,87,87,87,87,87,77,87,87,87
可转债基金,41,41,41,41,41,41,41,41,41,41,41,36,41,41,41
短期纯债型基金,55,55,54,55,55,55,55,55,55,55,55,54,55,55,55
被动指数型债券基金,17,17,17,17,17,17,17,17,17,17,17,11,17,17,17


## 1.1 Put it all totether

In [29]:
def create_bond_fund_info(trade_dt):
    trade_dt = dt.datetime.strptime(trade_dt, "%Y%m%d")
    date_3yrs_ago = trade_dt.replace(year=trade_dt.year - 3)
    report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
    current_date = trade_dt.strftime("%Y%m%d")
    
    security_ids = fd_alive_funds(report_dates_begin, 3)
    basic_info = fd_basicinfo(security_ids, current_date)
    type_info = fd_typeclass(security_ids, current_date)
    asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).fillna(0)
    asset_port["CONVBDRTO_TO_BDRTO"] = asset_port["CONVBDRTO"] / asset_port["BDRTO"] * 100
    last_portfolio = asset_port.groupby("SECURITYID").last()[["BDRTO"]]
    last_4_cov_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["CONVBDRTO_TO_BDRTO"]]
    last_4_stk_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["EQUITYINVERTO"]]

    df = pd.merge(basic_info, type_info, on="SECURITYID")
    df = pd.merge(df, last_portfolio, on="SECURITYID", how="left").drop_duplicates()
    df = pd.merge(df, last_4_stk_portfolio, on="SECURITYID", how="left").drop_duplicates()
    df = pd.merge(df, last_4_cov_portfolio, on="SECURITYID", how="left").drop_duplicates()
    
    df.loc[(df["BDRTO"] >= 80) & (df.L2NAME == "短期理财债券型基金"), "债券型（子类）"] = "短期纯债型基金"
    df.loc[(df["BDRTO"] >= 80) & (df["CONVBDRTO_TO_BDRTO"] >= 80), "债券型（子类）"] = "可转债基金"
    df.loc[(df["BDRTO"] >= 80) & (df["CONVBDRTO_TO_BDRTO"] >= 5) & (df["CONVBDRTO_TO_BDRTO"] < 80) & (df["EQUITYINVERTO"] <= 0.1), "债券型（子类）"] = "可投转债型债券基金"
    df.loc[(df["BDRTO"] >= 80) & (df["EQUITYINVERTO"] >= 0.1) & (~df["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "可投股票型债券基金"
    df.loc[(df["BDRTO"] >= 80) & (df["L2NAME"] == "指数债券型基金") & (~df["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "被动指数型债券基金"
    df.loc[pd.isnull(df["债券型（子类）"]), "债券型（子类）"] = "中长期纯债型基金"
    
    return df

In [30]:
%%time

df = create_bond_fund_info("20211111")#.groupby( "债券型（子类）")

Wall time: 7.81 s


In [32]:
%%time

create_bond_fund_info("20211111").groupby( "债券型（子类）").count()

Wall time: 6.12 s


,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,BDRTO,EQUITYINVERTO,CONVBDRTO_TO_BDRTO
债券型（子类）,,,,,,,,,,,,,,,
中长期纯债型基金,655,655,465,655,655,655,655,655,655,655,655,655,639,633,626
可投股票型债券基金,370,370,321,370,370,370,370,370,370,370,370,370,370,370,370
可投转债型债券基金,87,87,77,87,87,87,87,87,87,87,87,87,87,87,87
可转债基金,41,41,36,41,41,41,41,41,41,41,41,41,41,41,41
短期纯债型基金,55,55,54,55,55,55,55,55,55,55,55,55,55,55,54
被动指数型债券基金,17,17,11,17,17,17,17,17,17,17,17,17,17,17,17


# 2. 基金指数
--------------------

In [33]:
# 我们只计算指数的每日收益
# 在确定指数的基准日之后，可以直接使用收益计算指数的值。
# 使用 TQ_FD_DERIVEDN 获取基金的净值情况

start_dt = "2015-01-31"
final_trade_dt = "2021-11-09"
nav_type = "REPAIRUNITNAV"

rebalance_dates = [d.strftime("%Y%m%d") for d in makeSchedule(start_dt, final_trade_dt, tenor="3M", calendar="china.sse", dateRule=BizDayConventions.ModifiedFollowing)]

In [34]:
dfs = []

for i, trade_dt in enumerate(rebalance_dates[1:]):
    pre_trade_dt = rebalance_dates[i]
    print(pre_trade_dt, trade_dt)
    fund_info = create_bond_fund_info(trade_dt)
    pre_nav_info = fd_derieden(fund_info.SECURITYID.tolist(), pre_trade_dt).rename(columns={nav_type: "PRE" + nav_type})
    nav_info = fd_derieden(fund_info.SECURITYID.tolist(), trade_dt)

    total_df = pd.merge(fund_info, nav_info, on=["SECURITYID"])
    total_df = pd.merge(total_df, pre_nav_info, on=["SECURITYID"])
    total_df["chg."] = total_df[nav_type] / total_df["PRE" + nav_type] - 1.0
    res = total_df.groupby("债券型（子类）")[["chg."]].mean()
    dfs.append(res)

20150130 20150430
20150430 20150731
20150731 20151030
20151030 20160129
20160129 20160429
20160429 20160729
20160729 20161031
20161031 20170126
20170126 20170428
20170428 20170731
20170731 20171031
20171031 20180131
20180131 20180427
20180427 20180731
20180731 20181031
20181031 20190131
20190131 20190430
20190430 20190731
20190731 20191031
20191031 20200123
20200123 20200430
20200430 20200731
20200731 20201030
20201030 20210129
20210129 20210430
20210430 20210730
20210730 20211029
20211029 20211109


In [35]:
final_report = pd.concat(dfs, keys=rebalance_dates[1:]).reset_index()
final_report.pivot_table(index="level_0", columns="债券型（子类）", values="chg.").to_excel("030_债券型基金_bak.xlsx")

In [37]:
final_report.pivot_table(index="level_0", columns="债券型（子类）", values="chg.")

债券型（子类）,中长期纯债型基金,可投股票型债券基金,可投转债型债券基金,可转债基金,短期纯债型基金,被动指数型债券基金
level_0,,,,,,
20150430,0.044518,0.074199,0.030098,0.177746,NaN,0.018172
20150731,0.018413,-0.001703,0.012489,-0.168559,NaN,0.001508
20151030,0.019671,0.017910,0.022858,0.001486,NaN,0.023154
20160129,0.005979,-0.010265,0.013286,-0.132160,NaN,0.016452
20160429,0.001132,0.006828,0.000991,-0.011851,NaN,-0.000176
20160729,0.020778,0.020762,0.021270,-0.009703,NaN,0.020802
20161031,0.012197,0.015085,0.015368,0.041360,NaN,0.010548
20170126,-0.018752,-0.021641,-0.018342,-0.059164,NaN,-0.022313
20170428,-0.003381,-0.002405,-0.006429,-0.020709,NaN,-0.006358
